# Web scraping for cities information

## 1. Importing libraries

In [ ]:
from bs4 import BeautifulSoup
#!pip install --upgrade beautifulsoup4
#!pip install requests
import pandas as pd
import requests
import re
import sqlalchemy

## 2. Getting data for one city (Berlin)

In [ ]:
# find url and store it in a variable
url = "https://en.wikipedia.org/wiki/Berlin"

In [ ]:
# download html with a get request
headers = {'Accept-Language': 'en-US,en;q=0.8'}
response = requests.get(url, headers = headers)
response.status_code # 200 status code means OK!

In [ ]:
# parse html (create the 'soup')
wiki_soup_fr = BeautifulSoup(response.content, "html.parser")
# check that the html code looks like it should
wiki_soup_fr.prettify

In [ ]:
city_name = wiki_soup_fr.select("span.mw-page-title-main")[0].get_text()
country_name = wiki_soup_fr.select("table.infobox td.infobox-data a")[0].get_text()
latitude = wiki_soup_fr.select("span.latitude")[0].get_text()
longitude = wiki_soup_fr.select("span.longitude")[0].get_text()
# population_1 = wiki_soup_fr.select("td.infobox-data")[10].get_text()
population_2 = wiki_soup_fr.select_one('th.infobox-header:-soup-contains("Population")').parent.find_next_sibling().find(text=re.compile(r'\d+')) # better method


In [ ]:
# create a DataFrame
scooter_cities_df= pd.DataFrame(
    {"city": [city_name],
     "country": [country_name],
     "latitude": [latitude],
     "longitude": [longitude],
     "population": [population_2]
    }
)

## 3. Looping to add other cities

In [ ]:
list_of_cities = ["Frankfurt", "Hamburg", "Cologne", "Munich"] # input

for city in list_of_cities:

    url = "https://en.wikipedia.org/wiki/" + city
    #download html with a get request
    headers = {'Accept-Language': 'en-US,en;q=0.8'}
    response = requests.get(url, headers = headers)
    if response.status_code != 200: break  # 200 status code means OK!

    # parse html (create the 'soup')
    wiki_soup = BeautifulSoup(response.content, "html.parser")

    #extract name, country, latitude, longitude, population
    city_name = wiki_soup.select("span.mw-page-title-main")[0].getText()
    country_name = wiki_soup.select("table.infobox td.infobox-data")[0].getText()
    latitude = wiki_soup.select("span.latitude")[0].getText()
    longitude = wiki_soup.select("span.longitude")[0].getText()

    if wiki_soup.select_one('th.infobox-header:-soup-contains("Population")'):
        population = wiki_soup.select_one('th.infobox-header:-soup-contains("Population")').parent.find_next_sibling().find(text=re.compile(r'\d+'))

    #append information to the cities_df
    city_df = pd.DataFrame(
        {"city": [city_name],
         "country": [country_name],
         "latitude": [latitude],
         "longitude": [longitude],
         "population": [population]
        }
    )

    #scooter_cities_df = scooter_cities_df.append(city_df, ignore_index=True)
    scooter_cities_df = pd.concat([scooter_cities_df, city_df], ignore_index = True)

    # fixing latitude
    scooter_cities_df['latitude'] = scooter_cities_df['latitude'].str.split('″').str[0].str.replace('°', '.', regex=False).str.replace('′', '', regex=False)
    # fixing longitude
    scooter_cities_df['longitude'] = scooter_cities_df['longitude'].str.split('″').str[0].str.replace('°', '.', regex=False).str.replace('′', '', regex=False)

    # fixing population
    scooter_cities_df["population"] = scooter_cities_df["population"].str.replace(',', '', regex=False)

scooter_cities_df

## 4. Pushing data to MySQL

In [ ]:
schema="gans" 
host="wbs-project3-db.cunuvsto5hvy.us-east-1.rds.amazonaws.com"
user="admin"
password = "***"
port=3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

### cities data

In [ ]:
cities_sql_df = scooter_cities_df.loc[:, ['city', 'country']]
cities_sql_df['country'] = "DE"
cities_sql_df

In [ ]:
cities_sql_df.to_sql('cities', 
              if_exists='append', 
              con=con, 
              index=False)

### populations data

In [ ]:
populations_sql_df = scooter_cities_df.loc[:, ['population']]
populations_sql_df

In [ ]:
populations_sql_df.to_sql('populations', 
              if_exists='append', 
              con=con, 
              index=False)